In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/19 10:05
# @Author  : Wang Yujia
# @File    : data_extract_for_targets.ipynb
# @Description : 为了target datas提取信息

# 1. Preparations
## 1.1 全局设置
1. input的`data_path`来自“data_extract_for_PT.ipynb”

In [70]:
data_path = "../data/targets/target_data_original.csv"

# output path
target_output_path = "../data/targets/target_data_NP.csv"
target_data_key_path = "../data/targets/target_datakey.csv"

features = ['product_id', 'bidincrement', 'bidfee','retail']
import pandas as pd
import numpy as np
from visdom import Visdom
import csv

## 1.2 data 读取与清洗
1. 读取data，保存data_key

In [63]:
data = pd.read_csv(data_path, encoding="utf-8")
data.head()

,auction_id,product_id,item,desc,retail,price,finalprice,bidincrement,bidfee,winner,...,endtime_str,flg_click_only,flg_beginnerauction,flg_fixedprice,flg_endprice,N,cnt_uniq,cnt_n_2,P,T
0,87964,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,74.70,74.70,0.15,0.75,Cemo23,...,11:17 PDT 08-28-2008,0,0,0,0,498,21,1,0.047619,1128.266667
1,87965,10009881,psp-slim-lite-sony-piano-black,PSP Slim & Lite Sony Piano Black,169.99,83.10,83.10,0.15,0.75,Jacobsonnich,...,22:52 PDT 11-07-2008,0,1,0,0,554,21,1,0.047619,1128.266667
2,88638,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,19.65,19.65,0.15,0.75,Mokkis,...,22:02 PDT 08-23-2008,0,0,0,0,131,33,1,0.030303,3328.266667
3,88639,10006115,sony-ericsson-s500i-unlocked-mysterious-,Sony Ericsson S500i Unlocked Mysterious Green,499.99,47.10,47.10,0.15,0.75,Superloeffel,...,14:23 PDT 08-24-2008,0,0,0,0,314,33,1,0.030303,3328.266667
4,89434,10007148,ipod-touch-apple-8gb-with-software-upgra,iPod Touch Apple 8GB with Software Upgrade,299.99,63.45,63.45,0.15,0.75,02bigwill,...,10:56 PDT 08-22-2008,0,0,0,0,423,49,1,0.020408,1994.933333


In [61]:
data_key = data[features].copy()
data_key.drop_duplicates(inplace=True,ignore_index=True)
data_key.to_csv(target_data_key_path,header=True, encoding="utf-8",index=False)
print(f"target data中，一共包括 *{data_key.shape[0]}* 个data keys")
data_key.head()

target data中，一共包括 *1268* 个data keys


,product_id,bidincrement,bidfee,retail
0,10009881,0.15,0.75,169.99
1,10006115,0.15,0.75,499.99
2,10007148,0.15,0.75,299.99
3,10007263,0.15,0.75,89.99
4,10010575,0.15,0.75,59.99


## 1.3 functions about 'key'

In [62]:
# get key from i in 'data_key'
def get_key_from_index(i,flag="NotStr"):
    if(flag == "str"):
        key_i = list(data_key.iloc[i,:])
        key_i_str = (str(int(key_i[0])),str(key_i[1]),str(key_i[2]),str(key_i[3]))   # 'product_id'是int类型
        return key_i_str
    else:
        key_i = data_key.iloc[i,:]
        return key_i

def select_data_fromkey(key_i_str):
    return data[(data['product_id'] == key_i_str[0]) & (data['bidincrement'] == key_i_str[1]) & (data['bidfee'] == key_i_str[2]) & (data['retail'] == key_i_str[3])].copy()

# 2. 整理
## 2.1 N
1. 整理成一个[key]对应一个“N” vector的格式


In [64]:
N = {}
env_str = 'N_targets'
# viz = Visdom(env=env_str)
print("Done")

Done


In [65]:
# data_key.shape[0]
for i in range(0,data_key.shape[0]):
    # i = 0
    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)

    key_i_str = get_key_from_index(i,"str")         # key for N
    N[key_i_str] = np.array(data_i.loc[:,'N'])

print("Done")

Done


## 2.2 P
1. "P" vector也需要（画图），“N”是为了算MLE

In [ ]:
P = {}
env_str = 'P_targets'
# viz = Visdom(env=env_str)
print("Done")

In [69]:
# data_key.shape[0]
for i in range(0,data_key.shape[0]):
    # i = 0
    # get i_th data_key
    key_i = get_key_from_index(i)
    data_i = select_data_fromkey(key_i)
    data_i.reset_index(drop=True,inplace=True)
    data_i.sort_values(by='N',ignore_index=True,inplace=True)

    key_i_str = get_key_from_index(i,"str")         # key for P
    P[key_i_str] = np.array(data_i.loc[:,'P'])
print("Done")

Done


# 3 save
1. 保存结果以及viz环境

In [ ]:
# 保存整个环境
viz.save(envs=[env_str])

In [73]:

with open(target_output_path,"w+",encoding='utf—8',newline='') as f:
    w= csv.DictWriter(f,fieldnames=['product_id','bidincrement','bidfee','retail','N','P'])
    w.writeheader()

    key=list(N.keys())
    value_N =list(N.values())
    value_P =list(P.values())
    for i in range(len(key)):
        dic = {       #字典类型
            'product_id':key[i][0],
            'bidincrement':key[i][1],
            'bidfee':key[i][2],
            'retail':key[i][3],
            'N': list(value_N[i]),
            'P': list(value_P[i])
        }
        w.writerow(dic)   #数据写入csv文件
print("DONE")

DONE
